In [ ]:
import base64
import json
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

### Session

In [ ]:
import boto3, time, json

sess = boto3.Session()
sm = sess.client("sagemaker")
region = sess.region_name
account = boto3.client("sts").get_caller_identity().get("Account")

In [ ]:
pip install sagemaker

In [ ]:
wget https://s3.us-west-2.amazonaws.com/mar-files-ts0.4/flores_615.mar

### IAM Role

Note: make sure the IAM role has:

AmazonS3FullAccess

AmazonEC2ContainerRegistryFullAccess

AmazonSageMakerFullAccess

In [ ]:
import sagemaker

role = sagemaker.get_execution_role()
role

### Amazon Elastic Container Registry (ECR)

In [ ]:
registry_name = "dynalab-torchserve-sagemaker"

In [ ]:
image = f"{account}.dkr.ecr.{region}.amazonaws.com/{registry_name}:latest"


In [ ]:
image

In [ ]:
# you don't need to run the next cell as if it has already been registered

In [ ]:
!aws ecr create-repository --repository-name {registry_name}

### Pytorch Model Artifact (mar file)

In [ ]:
model_file_name = "flores_615"
sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()
prefix = 'Dyna'

In [ ]:
# no need to run next cell as if it has been already pushed to s3 bucket

In [ ]:
!tar cvfz {model_file_name}.tar.gz flores_615.mar
!aws s3 cp {model_file_name}.tar.gz s3://{bucket_name}/{prefix}/models/

In [ ]:
# model_artifact = 's3://sagemaker-us-west-2-664080794519/Dyna/models/flores_modified.tar.gz' # This should be changed to S3 path generated above
model_artifact = 's3://sagemaker-us-west-2-664080794519/Dyna/models/flores_615.tar.gz' # This should be changed to S3 path generated above
#model_artifact = 's3://sagemaker-us-west-2-664080794519/Dyna/models/flores3.tar.gz' # This should be changed to S3 path generated above



### Build a Customized TorchServe Docker container and push it to Amazon ECR

In [ ]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account}.dkr.ecr.{region}.amazonaws.com


In [ ]:
!docker build -t {registry_name} /home/ubuntu/dynalab/dynalab/dockerfiles/prod

In [ ]:
!docker tag {registry_name}:latest {image}

In [ ]:
!docker push {image}

In [ ]:
model_name = "flores3-torchserve-sagemaker"

### Create a Model for Sagemaker use

In [ ]:
container = {"Image": image, "ModelDataUrl": model_artifact}

create_model_response = sm.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=container
)
print(create_model_response["ModelArn"])


### Creating an Sagemaker endopoint

once an endpoint is registered, if you need to do any changes in the container or mar file, you can just do the modificaiton in steps where you make the contianer/ mar file and consequently delete the model and create new one. You won't need to delete/register a new endpoint. 

In [ ]:

import time

endpoint_config_name = "torchserve-endpoint-config-changed-" + time.strftime(
    "%Y-%m-%d-%H-%M-%S", time.gmtime()
)
print(endpoint_config_name)

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.g4dn.xlarge",
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

### Invoking the endpoint for inference

In [ ]:

endpoint_name = "flores-torchserve-endpoint-changed" + time.strftime(
    "%Y-%m-%d-%H-%M-%S", time.gmtime()
)
print(endpoint_name)

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

In [ ]:

%%time
resp = sm.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

### Batch transform jobs

In [ ]:
s3_bucket_name= 'sagemaker-us-west-2-664080794519'
batch_input = f"s3://{s3_bucket_name}/Dyna/batch_transform_flores_torchserve_sagemaker/"
batch_output = f"s3://{s3_bucket_name}/Dyna/batch_transform_flores_torchserve_sagemaker_output/"

In [ ]:
import time

batch_job_name = 'flores-batch' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
batch_job_name

In [ ]:
request = {
    "ModelClientConfig": {
        "InvocationsTimeoutInSeconds": 3600,
        "InvocationsMaxRetries": 1,
    },
    "TransformJobName": batch_job_name,
    "ModelName": model_name,
    "MaxConcurrentTransforms":1,
    "BatchStrategy": "MultiRecord",
    "TransformOutput": {"S3OutputPath": batch_output, "AssembleWith": "Line", "Accept": "application/json"},
    "TransformInput": {
        "DataSource": {
            "S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": batch_input}
        },
        
        "SplitType" : "Line",
        "ContentType": "application/json",
    },
    "TransformResources": {"InstanceType": "ml.p2.xlarge", "InstanceCount": 1},
}


In [ ]:
%%time
sm.create_transform_job(**request)

while True:
    response = sm.describe_transform_job(TransformJobName=batch_job_name)
    status = response["TransformJobStatus"]
    if status == "Completed":
        print("Transform job ended with status: " + status)
        break
    if status == "Failed":
        message = response["FailureReason"]
        print("Transform failed with the following error: {}".format(message))
        raise Exception("Transform job failed")
    print("Transform job is still in status: " + status)
    time.sleep(30)

#### Batch aggregated requests

In [ ]:
client = boto3.client("sagemaker")
client.delete_model(ModelName=model_name)


In [ ]:
client.delete_endpoint(EndpointName=endpoint_name)
client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)